<a href="https://colab.research.google.com/github/astrodevine/YB_Photometry/blob/master/PhotomStartUp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Defining classes and functions.")

In [ ]:
#Import all the things:

import plotly.graph_objects as go
import plotly as px
import numpy as np
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
plt.ion()
#get_ipython().run_line_magic('matplotlib', 'inline')
# some plots
#get_ipython().run_line_magic('matplotlib', 'qt')
# the interactive plot
from matplotlib.patches import Circle
from matplotlib.colors import LogNorm
#import astropy.units as u
from astropy.io import fits
from astropy import wcs
#from astropy.wcs import WCS
from astropy.io import ascii
#from astropy.coordinates import SkyCoord
#from astropy.coordinates import ICRS, Galactic, FK4, FK5
from astropy.visualization import make_lupton_rgb
#from astropy.modeling import models, fitting
from scipy import interpolate
#import itertools
import sys
import math
import csv
#import pylab as py
import copy
import cv2
import os
import pandas as pd
from astropy.nddata import Cutout2D

# These lines supress warnings
import warnings
warnings.filterwarnings('ignore')

#For beta testing "Pick up where you left off?" option
import pickle

#You will need to make sure the following packages have been installed first:

#from tkinter import *
#import tkinter
#conda install -c anaconda tk

#from photutils import centroid_com
#https://photutils.readthedocs.io/en/stable/install.html
#conda install photutils -c astropy

from itertools import chain, repeat
from ipywidgets import interact, interactive
import ipywidgets as widgets
#import threading
#import mynormalize
#import mycolorbar




In [ ]:
#interactive clicking and buttons
global cid
cid = 1

def donebuttonclick(_):
  fig.canvas.mpl_disconnect(cid)
  print('all done!')
  return

def onclick(event):
    global coords
    print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          ('double' if event.dblclick else 'single', event.button,
           event.x, event.y, event.xdata, event.ydata))
    plt.plot(event.xdata, event.ydata, 'r+', ',')
    fig.canvas.draw()
    coords.append((event.x, event.y))

def onclick2(event):
    global coords
    if event.button == 1:
      plt.plot(event.xdata, event.ydata, 'r+')
      fig.canvas.draw()
      coords.append((event.xdata, event.ydata))
      #print(coords)
    elif event.button == 3 and coords != []:
      plt.plot(coords[-1][0], coords[-1][1], color="black", marker='+')
      coords = coords[:-1]
      fig.canvas.draw()
      #print(coords)


donebutton_layout= widgets.Layout(width='auto', height='40px') #set width and height

donebutton = widgets.Button(
    description='CLICK HERE TO SUBMIT POINTS AND CONTINUE',
    disabled=False,
    display='flex',
    flex_flow='column',
    #align_items='center',
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    style=dict(font_weight='bold'),
    layout = donebutton_layout
)

donebutton.on_click(donebuttonclick)


In [ ]:
#function that shows image and collects the coordinates for interpolation, modified for Colab interactivity
def get_coords_colab(img, imgw, wave, ybid):
    #Plot the cropped image, this will be for user to draw polygon on
    global fig

    circle = Circle((dxw / 2, dyw / 2), YB_rad_pix, fill=False)
    circle1 = Circle((dxw / 2, dyw / 2), YB_rad_pix, fill=False)
    circle2 = Circle((dxw / 2, dyw / 2), YB_rad_pix, fill=False)
    circle3 = Circle((dxw / 2, dyw / 2), YB_rad_pix, fill=False)
    circle4 = Circle((dxw / 2, dyw / 2), YB_rad_pix, fill=False)

    %matplotlib ipympl
    fig = plt.figure(figsize=(10, 5))
    fig.canvas.header_visible = False
    ax1 = fig.add_subplot(1,3,1, title='24 + 8 um reference image')
    ax3 = fig.add_subplot(1,3,3, title='Directions')

    #Make sure to add this subplot last for the interactive redrawing etc to work correctly
    ax2 = fig.add_subplot(1,3,2, title='Select coordinates in this image') #,
                          # projection=imgw)
                          # uncomment above to add WCS labels; they were getting distracting

    fig.suptitle('You are examining the ' + wave + ' image for YB' +
                        str(ybid))

    global cid
    cid = fig.canvas.mpl_connect('button_press_event', onclick2)
    #print("CID is: ", cid)

    plotloc = []
    for i in range(0, len(YBloc)):
        if 0 <  abs(YBloc['l'][i] - YB_long_pix) < (dxw/2 - 5) and 0 < abs(YBloc['b'][i] - YB_lat_pix) < (dyw/2 -5):
          plotloc.append((YBloc['l'][i] - YB_long_pix + (dxw/2 - 5), YBloc['b'][i] - YB_lat_pix + (dyw/2 -5)))


    #Plot instructions in ax3
    ax3.axis([0, 10, 0, 10])
    ax3.axis('off')
    text = ("*Click in the grey-scale image. ")
    text1 = ("*Left click to add points.")
    text2 = ("*Right click to delete points.")
    text3 = ("*Cick the SUBMIT button to continue.")
    text4 = ("*Circle in reference image indicates MWP user radius.")
    text5 = ("*You will be prompted to inspect results in the next cell;")
    text6 = ("re-run the code in above cell to redo your selection.")
    ax3.text(1, 8, text, ha='left', wrap=True)
    ax3.text(1, 7, text1, ha='left', wrap=True)
    ax3.text(1, 6, text2, ha='left', wrap=True)
    ax3.text(1, 5, text3, ha='left', wrap=True)
    ax3.text(1, 4, text4, ha='left', wrap=True)
    ax3.text(1, 3, text5, ha='left', wrap=True)
    ax3.text(1, 2, text6, ha='left', wrap=True)


    #Plot rg image in ax1
    r = orig24
    g = orig8
    b = np.zeros(orig8.shape)
    ax1.axis('off')
    ax1.imshow(make_lupton_rgb(r, g, b, stretch=200, Q=0))#, norm=LogNorm())
    ax1.add_artist(circle)
    for point in range(0, len(plotloc)):
        ax1.plot(plotloc[point][0], plotloc[point][1], marker='+', markersize=40)
        circleloc = Circle((plotloc[point][0], plotloc[point][1]), YBloc['r'][point], fill = False, color = 'Blue')
        ax1.add_artist(circleloc)

    #Plot image of interest in ax2 for interactive clicking
    ax2.imshow(img, cmap='gray', norm=LogNorm())

    #Make button to exit when done
    display(widgets.HBox(children=[donebutton]), object_position='bottom center')

    #plt.pause(15)
    #return coords


In [ ]:
#YB/Source selection buttons and display grid

  #def YBselection1click():

In [ ]:
#flagging buttons
def flagbuttonclick1(change):
  global f1
  if change['new']:
    f1=1
  else: f1=0

def flagbuttonclick2(change):
  global f2
  if change['new']:
    f2=1
  else: f2=0

def flagbuttonclick3(change):
  global f3
  if change['new']:
    f3=1
  else: f3=0

def flagbuttonclick4(change):
  global f4
  if change['new']:
    f4=1
  else: f4=0

def flagbuttonclick5(change):
  global f5
  if change['new']:
    f5=1
  else: f5=0

def flagbuttonclick6(change):
  global f6
  if change['new']:
    f6=1
  else: f6=0

def flagbuttonclick7(change):
  global f7
  if change['new']:
    f7=1
  else: f7=0

def flagbuttonclick8(change):
  global f8
  if change['new']:
    f8=1
  else: f8=0

flayout=widgets.Layout(width='auto', height='40px')

Flag1button = widgets.ToggleButton(description = "Saturated image", value=False, disabled=False, layout=flayout)
Flag2button = widgets.ToggleButton(description = "Multiple sources in masked area", value=False, disabled=False, layout=flayout)
Flag3button = widgets.ToggleButton(description = "Filamentary structure(s)", value=False, disabled=False, layout=flayout)
Flag4button = widgets.ToggleButton(description = "No obvious source in image", value=False, disabled=False, layout=flayout)
Flag5button = widgets.ToggleButton(description = "IRDC association", value=False, disabled=False, layout=flayout)
Flag6button = widgets.ToggleButton(description = "Diffraction pattern/star", value=False, disabled=False, layout=flayout)
Flag7button = widgets.ToggleButton(description = "Poor confidence in photometry", value=False, disabled=False, layout=flayout)
Flag8button = widgets.ToggleButton(description = "Other", value=False, disabled=False, layout=flayout)

Flag1button.observe(flagbuttonclick1, 'value')
Flag2button.observe(flagbuttonclick2, 'value')
Flag3button.observe(flagbuttonclick3, 'value')
Flag4button.observe(flagbuttonclick4, 'value')
Flag5button.observe(flagbuttonclick5, 'value')
Flag6button.observe(flagbuttonclick6, 'value')
Flag7button.observe(flagbuttonclick7, 'value')
Flag8button.observe(flagbuttonclick8, 'value')

flaggrid=widgets.GridBox(children=[Flag1button, Flag2button, Flag3button, Flag4button, Flag5button, Flag6button, Flag7button, Flag8button], layout=widgets.Layout(width='100%', grid_template_rows='auto auto', grid_template_columns='25% 25% 25% 25%', grid_template_areas='"FlagButton1 FlagButton2 FlagButton3 FlagButton4", "FlagButton5 FlagButton6 FlagButton7 FlagButton8"'))

out=widgets.Output()

#Function to examine images and input flags for output file
def make_flags(fim1, fim2, um):
    plt.close('all')
    plt.figure(figsize=(6, 3))

    plt.suptitle("Select any flags to apply")
    plt.subplot(1, 2, 1, title='Original Data')
    plt.imshow(fim1,
               cmap='hot', norm=LogNorm())
               #norm=LogNorm(vmin=fim1.min(), vmax=fim1.max()))

    plt.subplot(1, 2, 2, title='Bkgrnd Removed')
    plt.imshow(fim2, cmap='hot')
    display(out)
    #clear previous flag input
    Flag1button.value = False
    Flag2button.value = False
    Flag3button.value = False
    Flag4button.value = False
    Flag5button.value = False
    Flag6button.value = False
    Flag7button.value = False
    Flag8button.value = False
    out.clear_output()
    with out:
      display(flaggrid)


In [ ]:
#function that generates and saves the four panel images to the folder photom_images
#call with (image, masked image, interpolated image, resid)
def make_figs(im1, im2, im3, im4, fitfile, imw, um):
    #print(im1.max(), im1.min())

    ############Generate the figures for each source##################
    #note I'm being lazy here and calling from the code things that aren't defined in function
    #this is pretty ugly and should maybe get cleaned up
    fig = plt.figure(figsize=(4, 4))

    #Plot the original image and MWP User YB circle
    circle = Circle((dxw / 2, dyw / 2), YB_rad_pix, fill=False)
    fig1 = plt.subplot(2, 2, 1, title='Cropped image', projection=imw)
    plt.imshow(im1, cmap='hot', norm=LogNorm())
    plt.axis('off')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    fig1.add_artist(circle)
    fig1.text(dxw / 2,
              dyw / 2 - 5,
              'MWP size',
              color='white',
              ha='center',
              va='top',
              weight='bold')

    #Plot the mask
    plt.subplot(2, 2, 2, title='Masked Image', projection=imw)
    plt.imshow(im2, cmap='hot', norm=LogNorm())
    plt.axis('off')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    #Plot the interpolated background
    plt.subplot(2, 2, 3, title='Interpolated image', projection=imw)
    plt.imshow(im3, cmap='hot', norm=LogNorm())
    plt.axis('off')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    #Plot the residual
    plt.subplot(2, 2, 4, title='Residual(Image-Interp)', projection=imw)
    plt.imshow(im4, cmap='hot')#, norm=LogNorm(vmin=im1.min(), vmax=im1.max()))
    plt.axis('off')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    #Make the plots pretty
    plt.tight_layout(pad=0.2, w_pad=5, h_pad=6)
    plt.subplots_adjust(top=0.9)
    fig.suptitle('Interpolation of YB %s at ' % (YB) + um, fontsize=10)

    plt.pause(1)

#saving the images feature disabled for online users, we don't want all those files!
#   # Save this result as a new png
#    figurename = os.path.join(path1,
#                              f"photom_images/{um}interpolation_YB_{YB}.png")
#    fig.savefig(figurename)
#
#    # Save the fits cut-outs for future use if needed
#    im1name = os.path.join(path1, f"fits_cutouts/{um}cropped_YB_{YB}.fits")
#    im2name = os.path.join(path1, f"fits_cutouts/{um}masked_YB_{YB}.fits")
#    im3name = os.path.join(path1, f"fits_cutouts/{um}interp_YB_{YB}.fits")
#    im4name = os.path.join(path1, f"fits_cutouts/{um}resid_YB_{YB}.fits")
#
#    fitfile.data=im1
#    fitfile.writeto(im1name, overwrite=True)
#
#    fitfile.data=im2
#    fitfile.writeto(im2name, overwrite=True)
#
#    fitfile.data=im3
#    fitfile.writeto(im3name, overwrite=True)
#
#    fitfile.data=im4
#    fitfile.writeto(im4name, overwrite=True)

In [ ]:
#class choose_image: use l range to choose and open right mosaics
# returns .um8, .um8data, .um8w - full info, data array, wcs header at 8 um
# returns .um12, .um12data, .um12w - full info, data array, wcs header at 12 um
# returns .um24, .um24data, .um124w - full info, data array, wcs header at 24 um
# GWC adding mosaics beyond the pilot region starting 02/22/22
class choose_image():
    def __init__(self, l, b):
        #currently don't need the WCS files for 12, 24 um because they
        #are reprojections onto 8um coordinate grid
        #GWC added 'b' on 4/5/22.

        if l >= 28.5 and l <= 31.5:
            path8 = os.path.join(path1,
                                 'GLM_03000+0000_mosaic_I4.fits')
            path12 = os.path.join(path1,
                                  'WISE_12um_03000_mosaic.fits')
            path24 = os.path.join(
                path1,
                'MIPSGAL_03000_mosaic_reprojected.fits')
            path70 = os.path.join(
                path1,
                'PACS_70um_03000_mosaic.fits')
        elif l > 31.5 and l <= 34.5:
            path8 = os.path.join(path1,
                                 'GLM_03300+0000_mosaic_I4.fits')
            path12 = os.path.join(path1,
                                  'WISE_12um_03300_mosaic.fits')
            path24 = os.path.join(
                path1,
                'MIPSGAL_03300_mosaic_reprojected.fits')
            path70 = os.path.join(
                path1,
                'PACS_70um_03300_mosaic.fits')
        elif l > 34.5 and l <= 37.5:
            path8 = os.path.join(path1,
                                 'GLM_03600+0000_mosaic_I4.fits')
            path12 = os.path.join(path1,
                                  'WISE_12um_03600_mosaic.fits')
            path24 = os.path.join(
                path1,
                'MIPSGAL_03600_mosaic_reprojected.fits')
            path70 = os.path.join(
                path1,
                'PACS_70um_03600_mosaic.fits')
        elif l > 37.5 and l <= 40.5:
            path8 = os.path.join(path1,
                                 'GLM_03900+0000_mosaic_I4.fits')
            path12 = os.path.join(path1,
                                  'WISE_12um_03900_mosaic.fits')
            path24 = os.path.join(
                path1,
                'MIPSGAL_03900_mosaic_reprojected.fits')
            path70 = os.path.join(
                path1,
                'PACS_70um_03900_mosaic.fits')
        elif l > 40.5 and l <= 43.5:
            path8 = os.path.join(path1,
                                 'GLM_04200+0000_mosaic_I4.fits')
            path12 = os.path.join(path1,
                                  'WISE_12um_04200_mosaic.fits')
            path24 = os.path.join(
                path1,
                'MIPSGAL_24um_04200_mosaic.fits')
            path70 = os.path.join(
                path1,
                'PACS_70um_04200_mosaic.fits')
        elif l > 43.5 and l <= 46.5:
            path8 = os.path.join(path1,
                                 'GLM_04500+0000_mosaic_I4.fits')
            path12 = os.path.join(path1,
                                  'WISE_12um_04500_mosaic.fits')
            path24 = os.path.join(
                path1,
                'MIPSGAL_24um_04500_mosaic.fits')
            path70 = os.path.join(
                path1,
                'PACS_70um_04500_mosaic.fits')
        elif l > 46.5 and l <= 49.5:
            path8 = os.path.join(path1,
                                 'GLM_04800+0000_mosaic_I4.fits')
            path12 = os.path.join(path1,
                                  'WISE_12um_04800_mosaic.fits')
            path24 = os.path.join(
                path1,
                'MIPSGAL_24um_04800_mosaic.fits')
            path70 = os.path.join(
                path1,
                'PACS_70um_04800_mosaic.fits')
        elif l > 49.5 and l <= 52.5:
            path8 = os.path.join(path1,
                                 'GLM_05100+0000_mosaic_I4.fits')
            path12 = os.path.join(path1,
                                  'WISE_12um_05100_mosaic.fits')
            path24 = os.path.join(
                path1,
                'MIPSGAL_24um_05100_mosaic.fits')
            path70 = os.path.join(
                path1,
                'PACS_70um_05100_mosaic.fits')
        elif l > 101.0 and l <= 105.59 and b < 3.06:
            path8 = os.path.join(path1,
                                 'SMOG_08um_10300_mosaic.fits')
            path12 = os.path.join(path1,
                                  'SMOG_12um_10300_mosaic.fits')
            path24 = os.path.join(
                path1,
                'SMOG_24um_10300_mosaic.fits')
            path70 = os.path.join(
                path1,
                'SMOG_70um_10300_mosaic.fits')
        elif l > 101.0 and l <= 105.59 and b >= 3.06:
            path8 = os.path.join(path1,
                                 'SMOG_08um_10300_mosaic.fits')
            path12 = os.path.join(path1,
                                  'SMOG_12um_10300_mosaic.fits')
            path24 = os.path.join(
                path1,
                'SMOG_24um_10300_mosaic_high_b.fits')
            path70 = os.path.join(
                path1,
                'SMOG_70um_10300_mosaic.fits')
        elif l > 105.59 and l <= 110.2:
            path8 = os.path.join(path1,
                                 'SMOG_08um_10700_mosaic.fits')
            path12 = os.path.join(path1,
                                  'SMOG_12um_10700_mosaic.fits')
            path24 = os.path.join(
                path1,
                'SMOG_24um_10700_mosaic.fits')
            path70 = os.path.join(
                path1,
                'SMOG_70um_10700_mosaic.fits')
        else:
            # GWC revised print statement from "outside the pilot..."
            print('Your YB is outside the region.')
            print('Please try again.')
            sys.exit()

        temp = fits.open(path8)[0]
        self.um8 = temp
        self.um8data = temp.data
        self.um8w = wcs.WCS(temp.header)
        temp = fits.open(path12)[0]
        self.um12 = temp
        self.um12data = temp.data
        self.um12w = wcs.WCS(temp.header)
        temp = fits.open(path24)[0]
        self.um24 = temp
        self.um24data = temp.data
        self.um24w = wcs.WCS(temp.header)
        temp = fits.open(path70)[0]
        self.um70 = temp
        self.um70data = temp.data
        self.um70w = wcs.WCS(temp.header)



In [ ]:

#class that does the masking and interpolation, returns masked, blanked, interpolated, and residual
class do_interp():
    def __init__(self, img, verts):

        #use the clicked values from the user to create a NaN mask
        vertices = np.array([verts], dtype=np.int32)
        xyvals = np.array(verts, dtype=np.int32)
        xmin = min(xyvals[:, 0]) - 5
        xmax = max(xyvals[:, 0]) + 5
        ymin = min(xyvals[:, 1]) - 5
        ymax = max(xyvals[:, 1]) + 5
        #print(xmin, xmax, ymin, ymax)
        mask = np.zeros_like(img)
        inverse_mask = np.zeros_like(img)
        region_mask = np.zeros_like(img)
        cutout = np.zeros_like(img)

        # filling pixels inside the polygon defined by "vertices" with the fill color
        cv2.fillPoly(mask, vertices, 255)
        #TURN ALL Non-ZERO to NaN
        inverse_mask[np.nonzero(mask)] = int(
            1)  # ones inside poly, zero outside

        mask[np.nonzero(mask)] = float('nan')
        #TURN ALL ZERO to 1
        mask[np.where(mask == 0)] = int(1)  # nan inside poly, 1 outside
        region_mask = mask
        region_mask = np.nan_to_num(region_mask)  # zero in poly, 1 outside
        cutout[ymin:ymax, xmin:xmax] = mask[ymin:ymax, xmin:xmax]
        #TURN EVERYTHING OUTSIDE THAT RANGE to NaN
        cutout[np.where(cutout == 0)] = float('nan')

        #TAKE image=workask*mask will make a image with original values but NaN in polygon
        #blank = img*mask
        blank = img * cutout
        self.masked = mask
        self.blanked = blank
        goodvals = np.where(np.isfinite(blank))

        #perform the interpolation over the masked coordinates
        x = goodvals[1]  # x values of finite coordinates
        y = goodvals[0]  # y values of finite coordinates

        for i in x:
            for j in y:

                def get_fvals(x, y):
                    range_array = np.arange(x.size)
                    vals = np.zeros(x.size)
                    for (i, xi, yi) in zip(range_array, x, y):
                        vals[i] = img[yi][xi]
                    return vals

        fvals = get_fvals(x, y)

        newfunc = interpolate.Rbf(
            x, y, fvals,
            function='multiquadric')  # the function that does interpolation
        allvals = np.where(img)  # whole region to interpolate over
        xnew = allvals[1]
        ynew = allvals[0]
        fnew = newfunc(xnew, ynew)

        #put the interpolated values back into a 2D array for display and other uses
        def make_2D(fnew, xnew, ynew, img):
            new_array = np.zeros(
                (int(xnew.size /
                     ((img.shape)[0])), int(ynew.size / ((img.shape)[1]))),
                dtype=float)
            #print(new_array)
            range_array = np.arange(fnew.size)
            #print("rangearay:",range_array)

            for (i, x, y) in zip(range_array, xnew, ynew):
                new_array[y][x] = fnew[i]

            return new_array

        fnew_2D = make_2D(fnew, xnew, ynew, img)

        self.interp = img * region_mask + fnew_2D * inverse_mask

        #generate the residual image (original - interpolated background)
        self.resid = img - (img * region_mask + fnew_2D * inverse_mask)




In [ ]:
#class that gets the flux from residual images. Unit conversions are applied here.
class get_flux_colab():
    def __init__(self, d, umsel):
        #reset the total flux value
        flux_tot = 0

        #go through 100 x 100 pixel residual image and add up the pixel values
        for ROW in range(0, 100):
            for column in range(0, 100):

                flux_tot = flux_tot + d[ROW][column]  #gives the value of photometry


            #convert units of flux total.  MIPS/IRAC in MJy/Sr*Pixel, want Jy
            #conversion: (MJy/Sr*Pixel)*(Sr/Pix)*(Jy/MJy)
            #WISE is in units of data number per pixel
            #Conversion: DN/Pix*Pixel*Jy/DN
        if umsel == 0:
          flux_tot = flux_tot * str_to_pix8 * 10**6
        elif umsel == 1:
          flux_tot = flux_tot * dn_to_jy12
        elif umsel == 2:
          flux_tot = flux_tot * str_to_pix8 * 10**6
        elif umsel == 3:
          flux_tot = flux_tot #No conversion necesary



        #print("total flux:")
        #print(flux_tot)

        self.flux = flux_tot
